In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:  tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:  print(e)

from PIL import Image
import numpy as np
import h5py
import matplotlib.pyplot as plt
import json
import cv2

from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

# Test Conv

In [16]:
C_in = 3
H_in = 5
W_in = 5
x = np.random.randint(0, 5, size=(C_in, H_in, W_in))    
x = x.astype(np.float32) 

np.savetxt('zz_x.txt', x.flatten(), fmt='%f', delimiter=' ')

In [12]:
tf.keras.backend.clear_session()

C_out = 2
K = 3

w = tf.keras.layers.Conv2D(C_out, (K, K), use_bias=False, input_shape=(C_in, H_in, W_in), data_format='channels_first')

In [18]:
y = w(np.expand_dims(x, axis=0))
y = y[0, :].numpy()
print(f"Shape y: {y.shape}")

np.savetxt('zz_y.txt', y.flatten(), fmt='%f', delimiter=' ')

Shape y: (2, 3, 3)


In [20]:
y

array([[[ 2.5325222 ,  1.6139797 ,  1.0411434 ],
        [ 1.6274841 ,  4.197127  ,  4.771555  ],
        [ 2.477254  ,  3.8891191 ,  1.4037782 ]],

       [[ 1.3302975 , -2.7124486 ,  0.13538164],
        [-0.31014547, -4.419247  , -1.1445129 ],
        [ 2.4885478 ,  0.525429  , -1.8610804 ]]], dtype=float32)

In [19]:
weight_matrix = tf.transpose(w.weights[0], perm=[3, 2, 0, 1]).numpy()
print(f"Shape weight_matrix: {weight_matrix.shape}")

np.savetxt('zz_w.txt', weight_matrix.flatten(), fmt='%f', delimiter=' ')

Shape weight_matrix: (2, 3, 3, 3)


# Test Dense

In [57]:
BATCH_SIZE = 1
n_in = 21632
n_out = 10

x = np.random.randint(0, 5, size=(BATCH_SIZE, n_in))    
x = x.astype(np.float32) 

np.savetxt('zz_x.txt', x[0].flatten(), fmt='%f', delimiter=' ')

In [64]:
tf.keras.backend.clear_session()

w = np.random.randint(0, 5, size=(n_in, n_out))
w = tf.convert_to_tensor(w, dtype=tf.float32)

b = np.random.randint(0, 5, size=(n_out))
b = tf.convert_to_tensor(b, dtype=tf.float32)

np.savetxt('zz_w.txt', w.numpy().flatten(), fmt='%f', delimiter=' ')
np.savetxt('zz_b.txt', b.numpy().flatten(), fmt='%f', delimiter=' ')

In [65]:
z = tf.matmul(x, w) + b  
print(f"Shape y: {y.shape}")

np.savetxt('zz_y.txt', y.flatten(), fmt='%f', delimiter=' ')

Shape y: (1, 10)


In [66]:
y

array([[85668., 85739., 86056., 86338., 87272., 85679., 85096., 85764.,
        85934., 85773.]])

# Test Flatten layer

In [97]:
x = np.random.randint(0, 5, size=(1, C_in, H_in, W_in))    
x.flatten()

array([3, 2, 4, 0, 3, 4, 3, 4, 2, 3, 0, 3, 1, 3, 3, 3, 1, 2, 2, 2, 4, 2,
       1, 0, 3, 2, 2, 0, 2, 1, 3, 1, 2, 1, 0, 4, 4, 3, 2, 1, 0, 4, 1, 0,
       3, 0, 3, 1, 1, 2, 2, 0, 4, 4, 4, 3, 1, 3, 4, 2, 4, 2, 1, 2, 3, 2,
       0, 0, 1, 2, 3, 1, 1, 4, 2])

In [98]:
class CustomFlatten(tf.keras.layers.Layer):
    def __init__(self):
        super(CustomFlatten, self).__init__()

    def call(self, inputs):
        return tf.reshape(inputs, [tf.shape(inputs)[0], -1])

In [99]:
y = CustomFlatten()(x)
print(f"Custom Flatten: {y}")
print()

Custom Flatten: [[3 2 4 0 3 4 3 4 2 3 0 3 1 3 3 3 1 2 2 2 4 2 1 0 3 2 2 0 2 1 3 1 2 1 0 4
  4 3 2 1 0 4 1 0 3 0 3 1 1 2 2 0 4 4 4 3 1 3 4 2 4 2 1 2 3 2 0 0 1 2 3 1
  1 4 2]]



In [78]:
y.shape

TensorShape([1, 75])

In [79]:
x

array([[[[3., 2., 4., 3., 3.],
         [2., 1., 3., 2., 1.],
         [4., 1., 2., 0., 4.],
         [3., 0., 3., 1., 4.],
         [0., 3., 0., 1., 2.]],

        [[4., 3., 0., 0., 0.],
         [1., 1., 4., 1., 2.],
         [2., 0., 3., 0., 4.],
         [2., 2., 3., 2., 1.],
         [0., 3., 2., 3., 2.]],

        [[2., 4., 4., 0., 2.],
         [0., 4., 2., 0., 2.],
         [4., 2., 3., 2., 1.],
         [4., 2., 2., 2., 1.],
         [2., 2., 4., 3., 4.]]]], dtype=float32)